# Summary

## Documents

- A Document is the unix of text that the LLM processes in order to answer queries. 
- A Document is not necessarly the whole file: files are typically chunked. Documents usually contain metadata including the name of the original document, the page from where the part of the document was extracted, author e.t.c.
- Full texts are usually split into document chunks to optimise LLm output. Larger chunks = more stuff in context = more risk of hallunication. 

## Embeddings

- LLMs store text as vectors in a high-dimensional space
- In this space, the position of each point (embedding) reflects the meaning of its corresponding text.
Just as similar words might be close to each other in a thesaurus, similar concepts end up close to each other in this embedding space.
- In simpler words: LLMs store text in "space". The way that text is store relative to each other captures the relationship between the text. So text close to each other might have similar meaning. Text at a higher vertical axis might describe parent chld relationship. 
 
## Vector Stores

- A vector store is a specialised database for storing and querying embeddings. It supports standard operations like create, read, update, and delete (CRUD).
- For retrieval, it allows searching for semantically similar text by comparing embedding vectors using similarity metrics such as cosine similarity.


## Tool Calling
- Tool calling enables an LLM to interact with systems e.g. calling an API or querying a database.
- When interacting with external tools, the request and response typically needs to confirm to a schema e.g. API Request Payload, SQL query structure.

## Structured Output

- 

## Few-shot Prompting
- Few-shot prompting is a technique used with large language models (LLMs) where you provide a small number of examples (typically 1–5) within the prompt to show the model how to perform a task
- Importantly, the examples should include "negatives" so that the LLM understands how to handle such cases.
- **Different chat model providers impose different requirements for valid message sequences**
